In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.feature_selection import VarianceThreshold
import random
import pickle
from sklearn.preprocessing import StandardScaler
import os
import tensorflow as tf
from tqdm.notebook import tqdm
from scipy.stats import multivariate_normal as mvn
import matplotlib.pyplot as plt
from scipy.linalg import block_diag
import warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

2023-07-10 17:40:08.945339: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-10 17:40:08.948186: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-10 17:40:08.988061: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-10 17:40:08.988788: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 17:40:09.986941: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# len(weights_ann_1[0].ravel())

In [4]:
def get_targets_with_weights(batch_data, initial_ensembles, size_ens): 
    
    target_dim = 1
    
    # weights_ann_1 = ann.get_weights()
    
    # h1  = ann.layers[1].output.shape[-1]

    n_hidden_1 = len(weights_ann_1[0].ravel())
    
    hidden_weights_1 = initial_ensembles[:,:n_hidden_1].reshape( size_ens, batch_data.shape[1], h1)
    
    
    hidden_output_1 = np.einsum('ij,kjl->kil', batch_data, hidden_weights_1)

    
    hidden_layer_bias_1 = initial_ensembles[:,n_hidden_1:(n_hidden_1 + h1)].reshape(size_ens, 1,  h1)


    hidden_output_1 = hidden_output_1 + hidden_layer_bias_1

    n_pred_weights_1 = len(weights_ann_1[2].ravel())

    output_weights_1 = initial_ensembles[:,(n_hidden_1 + h1):(n_hidden_1 + h1 + n_pred_weights_1) ].reshape(size_ens, h1, target_dim)


    output_1 = np.einsum('ijk,ikl->ijl', hidden_output_1, output_weights_1)


    output_layer_bias_1 = initial_ensembles[:,(n_hidden_1 + h1 + n_pred_weights_1):(n_hidden_1 + h1 + n_pred_weights_1 + target_dim)].reshape(size_ens, 1, target_dim)


    final_output_1 = output_1 + output_layer_bias_1
    
    final_output_1 = final_output_1[:,:, 0]
    
    # print(final_output_1.shape, initial_ensembles.shape)
    
    stack = np.hstack((final_output_1, initial_ensembles))

    
    return final_output_1, stack

In [5]:
def ann(hidden = 32, input_shape = 256, output_shape = 1): 
    input_layer = tf.keras.layers.Input(shape = (input_shape))
    hidden_layer = tf.keras.layers.Dense(hidden)
    hidden_output = hidden_layer(input_layer)
    pred_layer = tf.keras.layers.Dense(output_shape, activation = "relu")
    pred_output = pred_layer(hidden_output)
#     pred_output = tf.keras.layers.Activation("softmax")(pred_output)
    model = tf.keras.models.Model(input_layer, pred_output)
    return model

In [6]:
def generate_initial_ensembles(num_weights, lambda1, size_ens):
    mean_vec = np.zeros((num_weights,))
    cov_matrix = lambda1*np.identity(num_weights)
    mvn_samp = mvn(mean_vec, cov_matrix)
    return mvn_samp.rvs(size_ens)

In [7]:
def expit(x):
    """Compute softmax values for each sets of scores in x."""
#     e_x = np.exp(x - np.max(x))
    return 1 / (1 + np.exp(-x))

In [8]:
samp_ann =  ann(hidden = 16, input_shape = 32, output_shape = 1)

In [9]:
weights_ann_1 = samp_ann.get_weights()

In [10]:
h1  = samp_ann.layers[1].output.shape[-1]

In [11]:
h1

16

In [12]:
samp_ann.count_params()

545

In [13]:
hidden_neurons = h1

In [14]:
samp_ann_params = samp_ann.count_params()

In [15]:
from scipy.special import softmax

In [16]:
def get_initial_X_t(data1, data2, data3, data4, size_ens, var_weights = 1.0, var_weight_weights = 4.0, var_L = 1.0, var_D = 1.0):
    # samp_ann =  ann(hidden = hidden_neurons, input_shape = 32, output_shape = 1)
    
    initial_ensembles1 = generate_initial_ensembles(samp_ann_params, var_weights, size_ens)
    data1_out1, data1_stack1 = get_targets_with_weights(data1, initial_ensembles1, size_ens = size_ens)
    
    initial_ensembles2 = generate_initial_ensembles(samp_ann_params, var_weights, size_ens)
    data1_out2, data1_stack2 = get_targets_with_weights(data2, initial_ensembles2, size_ens = size_ens)
    
    initial_ensembles3 = generate_initial_ensembles(samp_ann_params, var_weights, size_ens)
    data2_out1, data2_stack1 = get_targets_with_weights(data3, initial_ensembles3, size_ens = size_ens)
    
    initial_ensembles4 = generate_initial_ensembles(samp_ann_params, var_weights, size_ens)
    data2_out2, data2_stack2 = get_targets_with_weights(data4, initial_ensembles4, size_ens = size_ens)   
    
    X_t = np.concatenate((np.expand_dims(data1_stack1, -1), np.expand_dims(data1_stack2, -1), 
                         np.expand_dims(data2_stack1, -1), np.expand_dims(data2_stack2, -1)), axis = -1)
    
    initial_ensembles_for_weights = generate_initial_ensembles(4, var_weight_weights, size_ens)
    initial_ensembles_for_weights = np.expand_dims(initial_ensembles_for_weights,1)
    
    # initial_ensembles_for_L = generate_initial_ensembles(4, var_L, size_ens)
    # initial_ensembles_for_L = np.expand_dims(initial_ensembles_for_L,1)    
    
    initial_ensembles_for_D1 = generate_initial_ensembles(1, var_D, size_ens).reshape(-1,1)
    # initial_ensembles_for_D2 = generate_initial_ensembles(1, var_D, size_ens).reshape(-1,1)
    
    initial_ensembles_for_D1_zero = np.zeros((size_ens,1,1)).reshape(-1,1)
    initial_ensembles_for_D2_zero = np.zeros((size_ens,1,1)).reshape(-1,1)
    
    initial_ensembles_for_D3_zero = np.zeros((size_ens,1,1)).reshape(-1,1)
    
    initial_ensembles_for_D = np.concatenate((np.expand_dims(initial_ensembles_for_D1,1),
                                                       np.expand_dims(initial_ensembles_for_D1_zero,1), 
                                                      np.expand_dims(initial_ensembles_for_D2_zero,1),
                                                       np.expand_dims(initial_ensembles_for_D3_zero,1)), axis = 2)
    
    # print(X_t.shape, initial_ensembles_for_weights.shape)
    
    X_t = np.concatenate((X_t, initial_ensembles_for_weights, initial_ensembles_for_D), axis = 1)
    
    initial_ensembles = np.hstack((initial_ensembles1, initial_ensembles2, initial_ensembles3, initial_ensembles4))
    
    return X_t, initial_ensembles, initial_ensembles_for_weights[:,0,:], initial_ensembles_for_D[:,0,:]

In [17]:
def get_weighted_targets_with_weights(batch_data, initial_ensembles, size_ens, weights): 
    
    target_dim = 1
    

    n_hidden_1 = len(weights_ann_1[0].ravel())
    
    hidden_weights_1 = initial_ensembles[:,:n_hidden_1].reshape( size_ens, batch_data.shape[1], h1)
    
    
    hidden_output_1 = np.einsum('ij,kjl->kil', batch_data, hidden_weights_1)

    
    hidden_layer_bias_1 = initial_ensembles[:,n_hidden_1:(n_hidden_1 + h1)].reshape(size_ens, 1,  h1)


    hidden_output_1 = hidden_output_1 + hidden_layer_bias_1

    n_pred_weights_1 = len(weights_ann_1[2].ravel())

    output_weights_1 = initial_ensembles[:,(n_hidden_1 + h1):(n_hidden_1 + h1 + n_pred_weights_1) ].reshape(size_ens, h1, target_dim)


    output_1 = np.einsum('ijk,ikl->ijl', hidden_output_1, output_weights_1)


    output_layer_bias_1 = initial_ensembles[:,(n_hidden_1 + h1 + n_pred_weights_1):(n_hidden_1 + h1 + n_pred_weights_1 + target_dim)].reshape(size_ens, 1, target_dim)


    final_output_1 = output_1 + output_layer_bias_1
    
    final_output_1 = final_output_1[:,:, 0]
    
    final_output_1 = final_output_1*weights
    
    # print(final_output_1.shape, initial_ensembles.shape)
    
    stack = np.hstack((final_output_1, initial_ensembles))

    
    return final_output_1, stack

In [18]:
from scipy.stats import beta

In [19]:
fudging_beta = beta(1,9)

In [20]:
def forward_operation(data1, data2, data3, data4, combined_ensembles , size_ens, fudging_beta):
    # samp_ann =  ann(hidden = hidden_neurons, input_shape = 32, output_shape = 1)
    params = samp_ann_params
    initial_ensembles1 = combined_ensembles[:, :params]
    initial_ensembles2 = combined_ensembles[:, params:(2*params)]
    initial_ensembles3 = combined_ensembles[:, (2*params):(3*params)]
    initial_ensembles4 = combined_ensembles[:, (3*params):(4*params)]

    
    initial_ensembles_for_weights = combined_ensembles[:, (4*params):(4*params + 4)]
    
    # initial_ensembles_for_L = combined_ensembles[:, (4*params + 4):(4*params + 4 + 4)]
    
    initial_ensembles_for_D = combined_ensembles[:,(4*params + 4):(4*params + 4 + 4 )]
    
    
    softmax_weights = softmax(initial_ensembles_for_weights)
    
    # +  fudging_beta.rvs(size_ens).reshape(-1,1)
    
    model_1 = softmax_weights[:, 0].reshape(-1,1) 
    
    # model_1 = np.min(model_1 -fudging_factor)
    
    model_2 = softmax_weights[:, 1].reshape(-1,1) 
    
    model_3 = softmax_weights[:, 2].reshape(-1,1) 
    
    model_4 = softmax_weights[:, 3].reshape(-1,1)
    
    sum_weights = model_1 + model_2 + model_3 + model_4
    
    
    # model_1_plus_model_2 = model_1 + model_2
    
    model_1 = model_1/sum_weights
    
    model_2 = model_2/sum_weights
    
    model_3 = model_3/sum_weights
    
    model_4 = model_4/sum_weights
    
    
    # print(np.mean(model_1 + model_2))
    
    data1_out1, data1_stack1 = get_weighted_targets_with_weights(data1, initial_ensembles1, size_ens = size_ens,
                                                                  weights=model_1)
    
    data1_out2, data1_stack2 = get_weighted_targets_with_weights(data2, initial_ensembles2, size_ens = size_ens,
                                                                weights=model_2)
    
    data2_out1, data2_stack1 = get_weighted_targets_with_weights(data3, initial_ensembles3, size_ens = size_ens,
                                                                 weights=model_3)
    
    data2_out2, data2_stack2 = get_weighted_targets_with_weights(data4, initial_ensembles4, size_ens = size_ens,
                                                                  weights=model_4)   
    
    X_t = np.concatenate((np.expand_dims(data1_stack1, -1), np.expand_dims(data1_stack2, -1), 
                         np.expand_dims(data2_stack1, -1), np.expand_dims(data2_stack2, -1)), axis = -1)
    
    initial_ensembles = np.hstack((initial_ensembles1, initial_ensembles2, initial_ensembles3, initial_ensembles4, 
                        initial_ensembles_for_weights, initial_ensembles_for_D))
    
    # print(X_t.shape)
    
    initial_ensembles_for_weights = np.expand_dims(initial_ensembles_for_weights,1)
    
    # initial_ensembles_for_L = np.expand_dims(initial_ensembles_for_L,1)
    
    initial_ensembles_for_D = np.expand_dims(initial_ensembles_for_D,1)
    
    # print(initial_ensembles_for_weights.shape)
    
    X_t = np.concatenate((X_t, initial_ensembles_for_weights, initial_ensembles_for_D), axis = 1)
    
    final_output = data1_out1 + data1_out2 + data2_out1 + data2_out2
    
    # weighted_psa = data1_out2 + data2_out2
    
    return X_t, initial_ensembles,final_output, model_1, model_2, model_3, model_4

In [21]:
def forward_operation_test(data1, data2, data3, data4, combined_ensembles , size_ens):
    # samp_ann =  ann(hidden = hidden_neurons, input_shape = 32, output_shape = 1)
    params = samp_ann_params
    initial_ensembles1 = combined_ensembles[:, :params]
    initial_ensembles2 = combined_ensembles[:, params:(2*params)]
    initial_ensembles3 = combined_ensembles[:, (2*params):(3*params)]
    initial_ensembles4 = combined_ensembles[:, (3*params):(4*params)]

    
    initial_ensembles_for_weights = combined_ensembles[:, (4*params):(4*params + 4)]
    
    # initial_ensembles_for_L = combined_ensembles[:, (4*params + 4):(4*params + 4 + 4)]
    
    initial_ensembles_for_D = combined_ensembles[:,(4*params + 4):(4*params + 4 + 4)]
    
    
    softmax_weights = softmax(initial_ensembles_for_weights)
    
    model_1 = softmax_weights[:, :1].reshape(-1,1)
    
    # model_1 = np.min(model_1 -fudging_factor)
    
    model_2 = softmax_weights[:, 1:2].reshape(-1,1) 
    
    model_3 = softmax_weights[:, 2:3].reshape(-1,1) 
    
    model_4 = softmax_weights[:, 3:4].reshape(-1,1)
    
    sum_weights = model_1 + model_2 + model_3 + model_4
    
    
    # model_1_plus_model_2 = model_1 + model_2
    
    model_1 = model_1/sum_weights
    
    model_2 = model_2/sum_weights
    
    model_3 = model_3/sum_weights
    
    model_4 = model_4/sum_weights
    
    data1_out1, data1_stack1 = get_weighted_targets_with_weights(data1, initial_ensembles1, size_ens = size_ens,
                                                                  weights=model_1)
    
    data1_out2, data1_stack2 = get_weighted_targets_with_weights(data2, initial_ensembles2, size_ens = size_ens,
                                                                weights=model_2)
    
    data2_out1, data2_stack1 = get_weighted_targets_with_weights(data3, initial_ensembles3, size_ens = size_ens,
                                                                 weights=model_3)
    
    data2_out2, data2_stack2 = get_weighted_targets_with_weights(data4, initial_ensembles4, size_ens = size_ens,
                                                                  weights=model_4)   
    
    X_t = np.concatenate((np.expand_dims(data1_stack1, -1), np.expand_dims(data1_stack2, -1), 
                         np.expand_dims(data2_stack1, -1), np.expand_dims(data2_stack2, -1)), axis = -1)
    
    initial_ensembles = np.hstack((initial_ensembles1, initial_ensembles2, initial_ensembles3, initial_ensembles4, 
                        initial_ensembles_for_weights, initial_ensembles_for_D))
    
    # print(X_t.shape)
    
    initial_ensembles_for_weights = np.expand_dims(initial_ensembles_for_weights,1)
    
    # initial_ensembles_for_L = np.expand_dims(initial_ensembles_for_L,1)
    
    initial_ensembles_for_D = np.expand_dims(initial_ensembles_for_D,1)
    
    # print(initial_ensembles_for_weights.shape)
    
    X_t = np.concatenate((X_t, initial_ensembles_for_weights, initial_ensembles_for_D), axis = 1)
    
    final_output = data1_out1 + data1_out2 + data2_out1 + data2_out2
    
    return X_t, initial_ensembles, final_output, model_1, model_2, model_3, model_4

In [22]:
total_weights = 4*(samp_ann.count_params() + 1 + 1)

In [23]:
reduction = 10

In [24]:
size_ens = total_weights//reduction

In [25]:
size_ens

218

In [26]:
G_t = [[1, 1, 1, 1]]
G_t = np.array(G_t).T

In [27]:
G_t.shape

(4, 1)

In [28]:
def get_predictions(data1, data2, data3, data4, initial_ensembles, fudging_beta  =fudging_beta): 
    _,_, weighted_alogp, w1, w2, w3, w4 = forward_operation(data1, data2, data3, data4, initial_ensembles, size_ens = size_ens, fudging_beta = fudging_beta)
    return weighted_alogp, w1, w2, w3, w4

In [29]:
def get_predictions_test(data1, data2, data3, data4, initial_ensembles): 
    _,_, weighted_alogp, w1, w2, w3, w4 = forward_operation_test(data1, data2, data3, data4, initial_ensembles, size_ens = size_ens)
    return weighted_alogp, w1, w2, w3, w4

In [30]:
def calculate_mu_bar_G_bar(data1, data2, data3, data4, initial_ensembles, fudging_beta):
    H_t = np.hstack((np.identity(data1.shape[0]), np.zeros((data1.shape[0], samp_ann_params + 1  + 1))))
    mu_bar = initial_ensembles.mean(0)
    X_t, _,_, _, _, _, _ = forward_operation(data1, data2, data3, data4, initial_ensembles, size_ens = size_ens, fudging_beta = fudging_beta)
    X_t = X_t.transpose((0,2,1))
    X_t = X_t.reshape(X_t.shape[0], X_t.shape[1]*X_t.shape[2])
    script_H_t = np.kron(G_t.T, H_t)
    G_u = (script_H_t@X_t.T)
    G_u = G_u.T
    G_bar = (G_u.mean(0)).ravel()
    return mu_bar.reshape(-1,1), G_bar.reshape(-1,1), G_u

In [31]:
def calculate_C_u(initial_ensembles, mu_bar, G_bar, G_u): 
    u_j_minus_u_bar = initial_ensembles - mu_bar.reshape(1,-1)
    G_u_minus_G_bar = G_u -  G_bar.reshape(1,-1)
    c = np.zeros((total_weights, G_bar.shape[0]))
    for i in range(0, size_ens): 
        c += np.kron(u_j_minus_u_bar[i, :].T.reshape(-1,1), G_u_minus_G_bar[i,:].reshape(-1,1).T)
    return c/size_ens, G_u_minus_G_bar

In [32]:
def calculate_D_u( G_bar, G_u): 
    G_u_minus_G_bar = G_u -  G_bar.reshape(1,-1)
    d = np.zeros((G_bar.shape[0], G_bar.shape[0]))
    for i in range(0, size_ens): 
        d += np.kron(G_u_minus_G_bar[i,:].T.reshape(-1,1), G_u_minus_G_bar[i,:].reshape(-1,1).T)
    return d/size_ens

In [33]:
def get_updated_ensemble(data1, data2, data3, data4, initial_ensembles, y_train, size_ens = size_ens, inflation_factor = 1.0, fudging_beta = fudging_beta, 
                        fudging_var = None):
    mu_bar, G_bar, G_u = calculate_mu_bar_G_bar(data1, data2, data3, data4, initial_ensembles, fudging_beta)
    C, G_u_minus_G_bar = calculate_C_u(initial_ensembles, mu_bar, G_bar, G_u)
    D = calculate_D_u( G_bar, G_u)
    _, R_t = create_cov(data1.shape[0],initial_ensembles)
    inflation = np.identity(R_t.shape[0])*inflation_factor
    D_plus_cov = D + (R_t *inflation_factor)
    D_plus_cov_inv = np.linalg.inv(D_plus_cov)
    mid_quant = C@D_plus_cov_inv
    noise_vec_mean = np.zeros((R_t.shape[0], ))
    noise_mvn = mvn(noise_vec_mean, R_t)
    fudging = noise_mvn.rvs(size_ens)
    interim = (y_train.T.flatten().reshape(1,-1) + fudging)
    right_quant = interim - G_u
    mid_times_right = mid_quant@right_quant.T
    updated_ensemble = (initial_ensembles + mid_times_right.T)
    if fudging_var is not None: 
        mean_vec = np.zeros((updated_ensemble.shape[1],))
        cov_mat = np.identity(updated_ensemble.shape[1])*fudging_var
        fudging_for_updated_ensembles = mvn(mean_vec, cov_mat)
        fudging_for_updated_ensembles_vec = fudging_for_updated_ensembles.rvs(size_ens)
        updated_ensemble = updated_ensemble + fudging_for_updated_ensembles_vec
    return updated_ensemble

In [34]:
target_dim = 1

In [35]:
lambda_D = 1

In [36]:
from joblib import Parallel, delayed

In [37]:
def softplus(x):
    return np.log(np.exp(x) + 1)

In [38]:
def create_cov(shape, initial_ensembles):
    cov_part = initial_ensembles[:, -4:-3]
    cov_part = cov_part.mean(0)
    variances1 = softplus(cov_part)
    n = shape
    return variances1, np.identity(n)*variances1

In [39]:
# idx = 1

In [40]:
with open("..//Data//gcn_cdr_train_pca.pickle", "rb") as f: 
    catch_train = pickle.load(f)

In [41]:
# catch_train[1]

In [42]:
with open("..//Data//gcn_cdr_test_pca.pickle", "rb") as f: 
    catch_test = pickle.load(f)

In [43]:
import random

In [44]:
def prepare_data_train(catch_train, size): 
    idxes = random.sample(range(0, catch_train[0].shape[0]), k = size)
    idxes = list(idxes)
    data1, data2, data3, data4 = catch_train[0][idxes,:], catch_train[1][idxes,:], catch_train[2][idxes,:], catch_train[3][idxes,:]
    
    y_train = catch_train[-1][idxes].reshape(-1,1)
    
    return data1, data2, data3, data4, y_train 

In [45]:
def prepare_data_test( catch_test, size): 
    idxes = random.sample(range(0, catch_test[0].shape[0]), k = size)
    idxes = list(idxes)
    data1, data2, data3, data4 = catch_test[0][idxes,:], catch_test[1][idxes,:], catch_test[2][idxes,:], catch_test[3][idxes,:]
    y_train = catch_test[-1][idxes].reshape(-1,1)
    return data1, data2, data3, data4, y_train

In [46]:
# data1_train, data2_train, data3_train, data4_train, y_train =  prepare_data_train(catch_train, size = 2500)

In [47]:
# data1_test, data2_test, data3_test, data4_test, y_test =  prepare_data_test(catch_test, size = 1000)

In [48]:
from scipy.stats import pearsonr

In [49]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [50]:
batch_size = 500

In [51]:
# data1_train.shape

In [52]:
#     data1_train, data2_train, data3_train, data4_train, y_train =  prepare_data_train(catch_train, size = 10000)

#     train_idxes = random.sample(range(0, data1_train.shape[0]), k = data1_train.shape[0])
    
#     train_chunks = list(chunks(train_idxes, batch_size))

In [53]:
# len(train_chunks[0])

In [54]:
# len(train_chunks)

In [55]:
def get_results(idx, var_weights = 1.0, var_weight_weights = 4.0, var_D = 1, inflation_factor = 1.6, fudging_beta = beta(1,19), 
               fudging_var = 1e-3, epochs = 30):
    
    # smiles_feats_train, rdkit_feats_train, smiles_feats_valid, rdkit_feats_valid, y_train, y_train_actual, y_valid, y_valid_actual, initial_ensembles  = prepare_data(idx, var_weights = var_weights, var_weight_weights =var_weight_weights, var_L = var_L, var_D = var_D)
    
    
    data1_train, data2_train, data3_train, data4_train, y_train =  prepare_data_train(catch_train, size = 5000)

    train_idxes = random.sample(range(0, data1_train.shape[0]), k = data1_train.shape[0])
    
    train_chunks = list(chunks(train_idxes, batch_size))
    
    data1_test, data2_test, data3_test, data4_test, y_test =  prepare_data_test(catch_test, size = 2500)
    
    X_t, initial_ensembles, initial_ensembles_for_weights, initial_ensembles_for_D = get_initial_X_t(data1_train, data2_train, data3_train, data4_train,
                                                                                                 size_ens = size_ens, var_weights = var_weights,
                                                                                                var_weight_weights = var_weight_weights,
                                                                                                     var_D = var_D)
    
    initial_ensembles = np.hstack((initial_ensembles, initial_ensembles_for_weights, initial_ensembles_for_D))
    
    best_rmse_train = 100000
    
    patience = 0
    
    for i in range(0,epochs):
        # print("epoch number is " +str(i))
        
        for chunk in train_chunks:
            
            data1_train1, data2_train1, data3_train1, data4_train1, y_train1 = data1_train[chunk,:], data2_train[chunk,:], data3_train[chunk,:], data4_train[chunk,:], y_train[chunk,:]

            initial_ensembles = get_updated_ensemble(data1_train1, data2_train1, data3_train1, data4_train1, initial_ensembles, y_train1, size_ens = size_ens,
                                                 inflation_factor = inflation_factor, fudging_beta = fudging_beta, fudging_var = fudging_var)
        
            G_u_train, w1, w2, w3, w4 = get_predictions(data1_train1, data2_train1, data3_train1, data4_train1, initial_ensembles, fudging_beta)
    
            li_train = np.percentile(G_u_train, axis = 0, q = (2.5, 97.5))[0,:].reshape(-1,1)    
            ui_train = np.percentile(G_u_train, axis = 0, q = (2.5, 97.5))[1,:].reshape(-1,1)  
    
            width_train = ui_train - li_train
            avg_width_train = width_train.mean(0)[0]
    
            ind_train = (y_train1 >= li_train) & (y_train1 <= ui_train)
            coverage_train= ind_train.mean(0)[0]
    
            averaged_targets_train = G_u_train.mean(0).reshape(-1,1)
            rmse_train = np.sqrt(((y_train1 -averaged_targets_train)**2).mean(0))[0]
        
            pearsonr_train = pearsonr(averaged_targets_train.reshape(averaged_targets_train.shape[0],), 
                                 y_train1.reshape(y_train1.shape[0],))
        
            r_train = pearsonr_train.statistic
    
            G_u_test, _, _, _, _ = get_predictions_test(data1_test, data2_test, data3_test, data4_test, initial_ensembles)
    

    
            li_test = np.percentile(G_u_test, axis = 0, q = (2.5, 97.5))[0,:].reshape(-1,1)     
            ui_test = np.percentile(G_u_test, axis = 0, q = (2.5, 97.5))[1,:].reshape(-1,1)   
    
            width_test = ui_test - li_test
            avg_width_test = width_test.mean(0)[0]
    
            ind_test = (y_test >= li_test) & (y_test <= ui_test)
            coverage_test= ind_test.mean(0)[0]
    
            averaged_targets_test = G_u_test.mean(0).reshape(-1,1)
            rmse_test = np.sqrt(((y_test -averaged_targets_test)**2).mean(0))[0]  
        
            pearsonr_test = pearsonr(averaged_targets_test.reshape(averaged_targets_test.shape[0],), 
                                 y_test.reshape(y_test.shape[0],))
        
            r_test = pearsonr_test.statistic

            # print("Training Coverage, Widths, RMSE, and Pearson R")
            # print(coverage_train, avg_width_train, rmse_train, r_train)
            # print("Testing Coverage, Widths, RMSE, and Pearson R")
            # print(coverage_test, avg_width_test, rmse_test, r_test)
            # print("Sum of Average Model Weights")
            # print(w1.mean() + w2.mean() + w3.mean() + w4.mean())
            # print("Sum of Average Model 1 Weights")
            # print(w1.mean(), w1.std())
            # print("Sum of Average Model 2 Weights")
            # print(w2.mean(), w2.std())
            # print("Sum of Average Model 3 Weights")
            # print(w3.mean(), w3.std())
            # print("Sum of Average Model 4 Weights")
            # print(w4.mean(), w4.std())

            if (rmse_train < best_rmse_train):
                best_pearsonr = r_train
                best_rmse_train = rmse_train
            # print("went here")
                best_train_width_mean = avg_width_train.mean()
                best_train_width = avg_width_train
            # best_smiles_weight = w1.mean()
                best_coverage_train = coverage_train
                best_rmse_train = rmse_train
                best_pearson_r = r_test
                best_test_width = avg_width_test

                best_coverage_test = coverage_test    
                best_rmse_test = rmse_test
                patience = 0
                best_ensembles = initial_ensembles
            
            else:
                patience = patience + 1
            
            # print("Patience is")
            # print(patience)
            # print('\n')
            
            if (patience > threshold) | (i == (epochs-1)):
            
            # print()
            # print(best_train_width.tolist(), best_coverage_train.tolist(), best_rmse_train.tolist(), best_test_width.tolist(), best_coverage_test.tolist(), best_rmse_test.tolist(), best_smiles_weight, flush = True)
            # print("done for fold" + str(idx), flush = True)
                print("train_coverage" + str(best_coverage_train), flush = True)
                print("test_coverage" + str(best_coverage_test), flush = True)
                print("train_width" + str(best_train_width), flush = True)
                print("test_width" + str(best_test_width), flush = True)
                print("pearson" + str(best_pearson_r), flush = True)
                print("rmse_train" + str(best_rmse_train), flush = True)
                print("rmse_test" + str(best_rmse_test), flush = True)
            # print("smiles_weight_ci" + str([best_li_smiles_weight, best_ui_smiles_weight]), flush = True)
            
                return [best_train_width, best_coverage_train, best_rmse_train, best_test_width, best_coverage_test, best_rmse_test, best_pearson_r, best_ensembles, [data1_test, data2_test, data3_test, data4_test, y_test]]


In [56]:
threshold = 10

In [57]:
# best_train_width, best_coverage_train, best_rmse_train, best_test_width, best_coverage_test, best_rmse_test, best_pearson_r, best_ensembles

In [58]:
from joblib import Parallel, delayed

In [59]:
%%time
get_results(0, var_weights = 3.0, var_weight_weights = 3.0, var_D = 1, inflation_factor =1, fudging_beta = beta(1,19), 
           fudging_var = 1e-2, epochs = 50)

train_coverage0.95
test_coverage0.902
train_width5.568277850363637
test_width5.461640131480348
pearson0.8400672347785835
rmse_train1.3252794080747985
rmse_test1.5376562433321106
CPU times: user 6min 24s, sys: 4min 9s, total: 10min 34s
Wall time: 1min 30s


[5.568277850363637,
 0.95,
 1.3252794080747985,
 5.461640131480348,
 0.902,
 1.5376562433321106,
 0.8400672347785835,
 array([[ 0.48334338, -0.99512841,  0.39591043, ..., -0.30359027,
         -0.8884931 , -0.07814189],
        [ 0.30378543, -1.18250323,  0.97320598, ..., -0.5084828 ,
         -1.38871392, -0.25467774],
        [ 0.00481866, -1.0408983 ,  0.97533784, ..., -0.56634768,
         -1.1075571 , -0.14250837],
        ...,
        [ 0.42899007, -0.87590464,  0.71549593, ..., -0.29716172,
         -1.08445111,  0.20729504],
        [ 0.58795435, -0.61705008,  0.73656062, ..., -0.18382558,
         -0.90743687, -0.11265691],
        [ 0.57179665, -0.68411289,  1.03875533, ..., -0.08806836,
         -0.89057015, -0.17749938]]),
 [array([[-31.0558849 ,  -7.69812794,   7.2790794 , ...,   6.60993213,
            1.34485188,   1.49409485],
         [  8.32017559,  17.57044508, -15.79252803, ...,  24.45345593,
            2.1559565 , -10.98377131],
         [ -8.02815969,  -2.3204692

In [60]:
catch_overall = Parallel(n_jobs = 14, backend = "loky", verbose = 8)(delayed(get_results)(idx,  var_weights = 3.0, var_weight_weights = 3.0, var_D = 1, inflation_factor =1, fudging_beta = beta(1,19), 
           fudging_var = 1e-2, epochs = 50) for idx in range(0,50))

[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.


train_coverage0.966
test_coverage0.9344
train_width6.608032484420482
test_width6.563397024665629
pearson0.8299583558779688
rmse_train1.3663738738380578
rmse_test1.6113758722191907
train_coverage1.0
test_coverage1.0
train_width30.60971448059724
test_width30.859966188268114
pearson0.7737809551773488
rmse_train1.8526858997565114
rmse_test1.866877290531824
train_coverage0.9
test_coverage0.842
train_width4.541081824243266
test_width4.604482851394375
pearson0.8373412088569717
rmse_train1.3522170477168507
rmse_test1.5966398481956383
train_coverage0.876
test_coverage0.8044
train_width4.2249423387065645
test_width4.133192462495216
pearson0.8219545506942935
rmse_train1.3160074606461805
rmse_test1.6091740159809609
train_coverage0.978
test_coverage0.9664
train_width7.288440986078596
test_width7.267751692319305
pearson0.8430914734534328
rmse_train1.3439381536490935
rmse_test1.554877152986873
train_coverage0.946
test_coverage0.938
train_width6.168589657041274
test_width6.138293370496912
pearson0.848

[Parallel(n_jobs=14)]: Done  13 tasks      | elapsed:  7.6min


train_coverage0.848
test_coverage0.8008
train_width4.0583735620286765
test_width4.144638582898742
pearson0.838667978297117
rmse_train1.3241922213482786
rmse_test1.5390028461231322
train_coverage0.994
test_coverage0.9776
train_width8.092412671749514
test_width8.117871158600787
pearson0.827290085079035
rmse_train1.4237815379654715
rmse_test1.6069752107674815
train_coverage0.986
test_coverage0.9548
train_width7.187179640168668
test_width7.290734016821086
pearson0.8295064151513025
rmse_train1.3223665572372574
rmse_test1.5749176689576627
train_coverage0.876
test_coverage0.8084
train_width4.281927519377705
test_width4.299112124021357
pearson0.826695530560799
rmse_train1.3045281740237755
rmse_test1.5964986086310893
train_coverage0.836
test_coverage0.7988
train_width3.966985037630957
test_width4.142578482661991
pearson0.8504656683812725
rmse_train1.3542250514553316
rmse_test1.5091225641954458
train_coverage1.0
test_coverage0.9976
train_width11.89748162338938
test_width11.93659328863632
pearson

[Parallel(n_jobs=14)]: Done  30 out of  50 | elapsed: 18.4min remaining: 12.3min


...

[Parallel(n_jobs=14)]: Done  37 out of  50 | elapsed: 22.3min remaining:  7.8min


train_coverage0.872
test_coverage0.8484
train_width4.617718249246711
test_width4.682201844759479
pearson0.8315816521450089
rmse_train1.3900364125900757
rmse_test1.5658173238918645
train_coverage0.962
test_coverage0.9416
train_width6.744350290844302
test_width6.624109314853002
pearson0.8238252525279334
rmse_train1.3180738179137441
rmse_test1.5696138641216562
train_coverage0.868
test_coverage0.8448
train_width4.645975721765668
test_width4.713544147122734
pearson0.8352164758646092
rmse_train1.3729714502071433
rmse_test1.5427485639280794
train_coverage0.96
test_coverage0.9388
train_width6.5861860353762935
test_width6.514607431416811
pearson0.8151236571204241
rmse_train1.3616962967297859
rmse_test1.600566949071685
train_coverage0.856
test_coverage0.8204
train_width4.510040392374692
test_width4.258101356195286
pearson0.8417563882448914
rmse_train1.3354474021241127
rmse_test1.5479450812774307
train_coverage0.958
test_coverage0.906
train_width6.047865491721992
test_width6.045875304200652
pears

[Parallel(n_jobs=14)]: Done  44 out of  50 | elapsed: 24.7min remaining:  3.4min


train_coverage0.92
test_coverage0.9104
train_width5.72661952396621
test_width5.7745551200292375
pearson0.8309018536957511
rmse_train1.4179733948546407
rmse_test1.5888390455158714
train_coverage0.806
test_coverage0.7772
train_width3.8280580784771447
test_width3.766947211325494
pearson0.8443492810751236
rmse_train1.3572820403975911
rmse_test1.503545417839696
train_coverage0.914
test_coverage0.866
train_width5.320851785118873
test_width5.259275697297069
pearson0.8345065960432101
rmse_train1.298300045476426
rmse_test1.5955625251374017
train_coverage0.884
test_coverage0.814
train_width4.355276332866703
test_width4.421617391468295
pearson0.8358444910381956
rmse_train1.290470929413374
rmse_test1.5450921556949264
train_coverage0.802
test_coverage0.7732
train_width3.8411690231890305
test_width3.8302938983335464
pearson0.8501796300237906
rmse_train1.3842297491411149
rmse_test1.5201527445395586
train_coverage0.844
test_coverage0.7908
train_width4.255603118686237
test_width4.137228740490755
pearso

[Parallel(n_jobs=14)]: Done  50 out of  50 | elapsed: 27.6min finished


In [61]:
with open("..//Data//catch_overall_50_reps_config_parallel_trial1_more_data.pickle", "wb") as f: 
    pickle.dump(catch_overall,f)

In [62]:
overall_df = pd.DataFrame(catch_overall).iloc[:,:7]

In [63]:
overall_df.mean()

0    6.794498
1    0.922200
2    1.358945
3    6.832834
4    0.892504
5    1.574794
6    0.831946
dtype: float64

In [64]:
overall_df.columns = ['best_train_width', 'best_coverage_train', 'best_rmse_train',
                      'best_test_width', 'best_coverage_test', 'best_rmse_test', 'best_pearson_r']

In [65]:
overall_df.mean()

best_train_width       6.794498
best_coverage_train    0.922200
best_rmse_train        1.358945
best_test_width        6.832834
best_coverage_test     0.892504
best_rmse_test         1.574794
best_pearson_r         0.831946
dtype: float64